In [1]:
from ltlcross_wrapper import Modulizer
from tools import sem_pipelines
from owl_servers import OwlServerManager, get_ldba_pipeline
import datetime

We use the client-server mode of Owl. This prevents Owl to start JVM for each command separately. Instead, we start one server for each configuration. These servers listen on ports (in our case) 6060 and 6061 which we then use to call these servers using `owl-client` command. The scripts to start and terminate servers, together with scripts to configure Owl, are in [owl_servers.py](owl_servers.py).

For each configuration we actually run 2 runs: One with Owl alone (`no.owl#variant`), the other (`yes.owl#variant`) uses Spot to simplfy the results using `autfilt`.

In [2]:
owl_m = OwlServerManager()
owl_m.new_server("owl#s", get_ldba_pipeline(mode="s"))
owl_m.new_server("owl#a", get_ldba_pipeline(mode="a"))


print(owl_m.get_clients().items())
owl_pipelines = {}
for mode, cmd in owl_m.get_clients().items():
    owl_pipelines[f"yes.{mode}"] = f"{cmd} | autfilt --small --tgba > %O"
    owl_pipelines[f"no.{mode}"] = f"{cmd} > %O"

dict_items([('owl#s', 'owl-client localhost 6060 %f'), ('owl#a', 'owl-client localhost 6061 %f')])


We take the Seminator configurations from the file [tools.py](tools.py). The printed values of `tools` are of the form `name_of_the_configuration : ltlcross_command`. This dict is then passed to `Modulizer` from `ltlcross_wrapper` for parallel execution of `ltlcross`.

In [3]:
tools = sem_pipelines.copy()
# tools.update(owl_pipelines)
for name, cmd in tools.items():
    print(f"{name}: {cmd}")

yes.seminator#slim: ltl2tgba --deterministic -f %f | seminator --slim --pure --postprocess=1 | autfilt --small --tgba > %O
yes.seminator#weakslim: ltl2tgba --deterministic -f %f | seminator --slim --weak --pure --postprocess=1 | autfilt --small --tgba > %O
yes.empc#specialslim: docastom=`mktemp`;docas=`mktemp`;ltl2tgba --deterministic -f %f>$docastom; java -jar ~/automaty/epmc-ashu-v11.jar formula2automaton --ashu-input-type hoa --ashu-input-file $docastom --ashu-slim-exploration-nonempty-breakpoint true  --ashu-automaton-type slim --ashu-output-file $docas;autfilt --small --tgba -F $docas > %O
yes.empc#slim: docastom=`mktemp`;docas=`mktemp`;ltl2tgba --deterministic -f %f>$docastom; java -jar ~/automaty/epmc-ashu-v11.jar formula2automaton --ashu-input-type hoa --ashu-input-file $docastom   --ashu-automaton-type slim --ashu-output-file $docas;autfilt --small --tgba -F $docas > %O
yes.empc#specialslim#acc: docastom=`mktemp`;docas=`mktemp`;ltl2tgba --deterministic -f %f>$docastom; java -j

In [4]:
print("started ", datetime.datetime.now().strftime("%H:%M:%S"))
!mkdir -p data
for s in ["literature", "random"]:
    m = Modulizer(tools, f"../formulae/{s}_nd.ltl", 
                  name=f"{s}_nd", tmp_dir=f"data/{s}_nd.parts",
                  processes=6, timeout="30")
    m.run()
    !mv {s}_nd* data
print("finished ", datetime.datetime.now().strftime("%H:%M:%S"))

started  14:29:18
finished  15:21:31


In [ ]:
for s in ["literature","random"]:
    for t in ["det","sd"]:
        name = f"{s}_{t}"
        m = Modulizer(tools, f"../formulae/{name}.ltl", 
                      name=name, tmp_dir=f"data/{name}.parts",
                      processes=6, timeout="30")
        m.run()
        !mv {name}* data

In [ ]:
owl_m.terminate_all()